In [3]:
import os


In [7]:
from pathlib import Path

abs_dir = os.path.abspath(".")
dir_path = Path(abs_dir)
dir_list = ["store", "ts"]


def prepare():
    for d in dir_list:
        if not os.path.exists(d):
            os.mkdir(dir_path / d)


prepare()

In [1]:

from typing import Optional
import yaml

del_list = ["prepare-", "-prepare", ""]


class Download:
    MAX_THREAD_NUM = 20

    def __init__(self):
        self.fail_count = 0
        self.name = None
        self.setup()

    def setup(self):
        pass

    @staticmethod
    def download(self):
        pass

    @staticmethod
    def get_key_url(m3u8_url, m3u8_text):
        key_url = None
        res = re.search('URI=".*"', m3u8_text)
        if res:
            key_url_part = res.group()[5:-1]
            if "http" in key_url_part:
                return key_url_part
            for i in range(1, 10):
                m3u8_url_part = "".join(m3u8_url.rsplit("/", i)[0])
                key_url = m3u8_url_part + '/' + key_url_part
                response = requests.get(key_url)
                if response.status_code == 200:
                    break
            return key_url
        else:
            return None

    def get_key(self, m3u8_url, m3u8_text):
        key_url = self.get_key_url(m3u8_url, m3u8_text)
        if key_url is None:
            raise Exception("key url not found or not accessible")
        res = requests.get(key_url)
        return res.content

    @staticmethod
    def get_iv(m3u8_text) -> Optional[bytes]:
        res = re.search("IV=.*\n", m3u8_text)
        if res:
            return bytes.fromhex(res.group()[5:-1])
        else:
            return None

    def download_by_m3u8(self, m3u8_url, m3u8_text):
        def deal_url(url):
            url = url.split(".m3u8")[0] + ".m3u8"
            for s in del_list:
                url = url.replace(s, '')
            return url

        m3u8_url = deal_url(m3u8_url)

        with open("record.yaml", 'r') as f:
            record = yaml.safe_load(f)
        if m3u8_url in record.keys():
            name = record[m3u8_url]
            if name in os.listdir("store"):
                print("downloaded before")
                return

        dir_list = []
        for d in os.listdir("store"):
            if os.path.isdir("store/" + d):
                dir_list.append(d)
        dir_list.sort(key=int)
        if dir_list:
            name = str(int(dir_list[-1]) + 1)
        else:
            name = "1"
        os.mkdir("store/" + name)
        self.name = name

        with open("store/" + self.name + '/' + "url.txt", 'w') as f:
            f.write(m3u8_url)

        record[m3u8_url] = self.name
        with open("record.yaml", 'w') as f:
            yaml.dump(record, f)

        self._download_ts(m3u8_url, m3u8_text)

    def _download_ts(self, m3u8_url, m3u8_text):
        ts_url_part_list = re.findall(".*.ts", m3u8_text)
        print("ts_url_part_list:", ts_url_part_list)
        test_part = ts_url_part_list[0]

        m3u8_url_part = ''
        if "http" not in test_part:
            for i in range(1, 10):
                m3u8_url_part = "".join(m3u8_url.rsplit("/", i)[0])
                url = m3u8_url_part + '/' + test_part
                response = requests.get(url)
                if response.status_code == 200:
                    break
            m3u8_url_part = m3u8_url_part + '/'
            print("m3u8_url_part:", m3u8_url_part)

        ts_head = ''
        for i in range(1, len(test_part) + 1):
            url = m3u8_url_part + test_part[:-3 - i] + '0.ts'
            response = requests.get(url)
            if response.status_code == 200:
                ts_head = test_part[:-3 - i]
            elif response.status_code == 404:
                break
            else:
                raise Exception("net error")

        ts_url_part = m3u8_url_part + ts_head
        print("ts_url_part", ts_url_part)

        key = self.get_key(m3u8_url, m3u8_text)
        iv = self.get_iv(m3u8_text)
        print(f"key: {key}, iv: {iv}")
        cipher = AES.new(key, AES.MODE_CBC, iv)

        self.fail_count = 0

        def download_ts(url, out):
            response = requests.get(url, out)
            if response.status_code == 200:
                print(url)
                import time
                print(time.ctime())
                with open(out, "wb") as f:
                    content = cipher.decrypt(response.content)
                    f.write(content)
            else:
                self.fail_count += 1

        thread_list = []
        for i in range(10000):
            url = ts_url_part + str(i) + ".ts"
            out = "store/" + self.name + '/' + str(i) + ".ts"
            if len(thread_list) <= self.MAX_THREAD_NUM:
                thread = Thread(target=download_ts, args=(url, out))
                thread_list.append(thread)
            else:
                thread = Thread(target=download_ts, args=(url, out))
                thread_list.append(thread)
                for thread in thread_list:
                    thread.start()
                for thread in thread_list:
                    thread.join()
                thread_list.clear()
            if self.fail_count > 30:
                break
        print(f"download ok, downloaded {len(os.listdir('store/' + self.name))}")

        for file in os.listdir("store/" + self.name):
            with open("ts/" + self.name + ".ts", "wb") as l_f:
                with open("store/" + self.name + '/' + file, "rb") as s_f:
                    l_f.write(s_f.read())

    @staticmethod
    def decode_ts(encoded_data: str, key: bytes, iv: bytes = None):
        cipher = AES.new(key, AES.MODE_CBC, iv)
        plain_data = cipher.decrypt(encoded_data)
        return plain_data


download = Download()

m3u8_url = input("url: ")
with open("m3u8.m3u8", 'r') as f:
    m3u8_text = f.read()
    download.download_by_m3u8(m3u8_url, m3u8_text)

FileNotFoundError: [Errno 2] No such file or directory: 'm3u8.m3u8'

In [13]:
import os

with open("ts/1" + ".ts", "wb") as l_f:
    file_list = os.listdir("store/1")
    file_list.sort(key=lambda x: int(x.replace(".ts", "")) if ".ts" in x else 0)
    for file in file_list:
        if ".ts" in file:
            print(file)
            with open("store/1" + '/' + file, "rb") as s_f:
                l_f.write(s_f.read())

0.ts
1.ts
2.ts
3.ts
4.ts
5.ts
6.ts
7.ts
8.ts
9.ts
10.ts
11.ts


In [1]:
"https://oss.fstsc.xyz/20221020/BtOeZi79/index.m3u8"
"https://oss.fstsc.xyz/20221124/jKE35AmG/index.m3u8"
"https://oss.fstsc.xyz/20221019/rNIRLkA7/5121kb/hls/index.m3u8"
"https://oss.fstsc.xyz/20230116/RAlcaGBG/3344kb/hls/index.m3u8"
"https://oss.fstsc.xyz/20230113/UHJSRPO3/index.m3u8"
"https://oss.fstsc.xyz/20230122/t1dRT4M0/3353kb/hls/index.m3u8"
"https://oss.fstsc.xyz/20221226/aIJn0XR9/4023kb/hls/index.m3u8"

SyntaxError: EOL while scanning string literal (2583625225.py, line 7)

In [3]:
import wget

wget.download("https://oss.fstsc.xyz/20221020/BtOeZi79/4065kb/hls/index.m3u8", "./test.m3u8")

'./test (1).m3u8'

In [4]:
wget.download("https://oss.fstsc.xyz/20221020/BtOeZi79/4065kb/hls/1HAqreFn.jpg", "./test.jpg")

'./test.jpg'

In [5]:
head = "https://oss.fstsc.xyz"

In [6]:
key_url = head + "/20221020/BtOeZi79/4065kb/hls/key.key"
wget.download(key_url, "./key.key")

'./key.key'

In [31]:
from threading import Thread
import os
import re
import shutil
from Crypto.Cipher import AES
import requests

m3u8_url = "https://oss.fstsc.xyz/20221020/BtOeZi79/index.m3u8"
head = ''
name = ''
key = ''
ts_name_list = []


def get_url(ord_url):
    global head
    head = "/".join(ord_url.split("/", 3)[:3])

    text = requests.get(ord_url).text
    text = text.split('\n')
    for l in text:
        if ".m3u8" in l:
            return head + l


def decode_and_download(url, path):
    encoded_data = requests.get(url).content
    cipher = AES.new(key, AES.MODE_CBC, b"0000000000000000")
    data = cipher.decrypt(encoded_data)
    with open(path, 'wb') as f:
        f.write(data)


def download(url):
    dir_list = os.listdir("ts")
    dir_list.sort(key=lambda x: int(x.replace(".ts", '')))
    global name
    if dir_list:
        name = str(int(dir_list[-1].replace(".ts", '')) + 1)
    else:
        name = '1'

    MAX_THREAD_NUM = 20
    text = requests.get(url).text
    text_lines = text.split('\n')
    key_part = None
    for l in text_lines:
        if ".key" in l:
            res = l.find("URI=")
            key_part = l[res + 5:-1]
    global key
    key = requests.get(head + key_part).content

    thread_list = []
    for l in text_lines:
        if len(thread_list) <= MAX_THREAD_NUM:
            if ".jpg" in l:
                l = l.replace(".jpg", '.ts')
                ts_name = l.rsplit('/', 1)[-1]
                ts_name_list.append(ts_name)
                thread = Thread(target=decode_and_download, args=((head + l), "store/" + name + '/' + ts_name))
                thread_list.append(thread)
        else:
            # for thread in thread_list:
            #     thread.start()
            # for thread in thread_list:
            #     thread.join()
            thread_list.clear()
    with open("ts/" + name + '.ts') as ts:
        for ts_name in ts_name_list:
            path = "store/" + name + '/' + ts_name
            with open(path, "wb") as f:
                ts.write(f.read())
    shutil.rmtree("store")


url = get_url(m3u8_url)
download(url)

"/20221020/BtOeZi79/4065kb/hls/key.key


['download.ipynb', 'download.py', 'key.key', 'record.yaml', 'store', 'test (1).m3u8', 'test.ipynb', 'test.py', 'test.ts', 'ts']
